In [1]:
using Pkg: @pkg_str

pkg"activate .."
pkg"add BenchmarkTools"
pkg"status"

Activating environment at `~/JuliaEnvs/Arborist/Project.toml`
  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
  Updating registry at `~/.julia/registries/Invenia`
  Updating git-repo `git@gitlab.invenia.ca:invenia/PackageRegistry.git`
 Resolving package versions...
  Updating `~/JuliaEnvs/Arborist/Project.toml`
 [no changes]
  Updating `~/JuliaEnvs/Arborist/Manifest.toml`
 [no changes]
Project Arborist v0.1.0
    Status `~/JuliaEnvs/Arborist/Project.toml`
  [6e4b80f9] + BenchmarkTools v0.4.2
  [da1fd8a2] + CodeTracking v0.5.7
  [1914dd2f] + MacroTools v0.5.1
  [295af30f] + Revise v2.1.6
  [b77e0a4c] + InteractiveUtils 
    Status `~/JuliaEnvs/Arborist/Manifest.toml`
  [6e4b80f9] + BenchmarkTools v0.4.2
  [da1fd8a2] + CodeTracking v0.5.7
  [1914dd2f] + MacroTools v0.5.1
  [295af30f] + Revise v2.1.6
  [b77e0a4c] + InteractiveUtils 


In [26]:
using Arborist
using Arborist: graft, get_ast
using Test
using BenchmarkTools
using MacroTools

In [27]:
includet("../test/demo.jl")

In [59]:
function fma_grafter(ast)
    MacroTools.postwalk(ast) do expr
        if (
            @capture(expr, x_ * y_ + z_) ||
            @capture(expr, z_ + x_ * y_)        
            )
            return :(fma($x, $y, $z))
        elseif isexpr(expr, :call)
            return Expr(:call,
                :graft, fma_grafter, expr.args...
            )
        end
        return expr
    end
end

Arborist.redeclare_graft()

graft (generic function with 3 methods)

In [61]:
bigmat = rand(Float16, (100,100))

graft(fma_grafter, fma_demo2, bigmat) |> extrema

(Array{Float16, 2}, Float16)
(Array{Float16, 2}, Float16)


ErrorException: no unique matching method found for the specified argument types

In [51]:
fma_demo2(bigmat) |> extrema

(Float16(138.5), Float16(259.8))

In [52]:
fma_demo2(bigmat)

100×100 Array{Float16,2}:
 185.4  190.6  209.2  192.1  205.1  …  208.8  211.8  226.5  212.8  202.9
 184.4  183.8  202.6  190.5  199.9     212.9  190.9  221.1  183.5  199.0
 168.2  184.0  184.8  167.8  180.6     197.8  188.1  206.6  184.0  184.9
 176.9  190.8  194.2  188.4  184.5     211.9  207.1  224.9  186.2  195.5
 171.8  158.1  181.0  174.5  191.8     192.1  183.4  199.8  181.6  182.6
 165.2  182.6  187.1  168.4  189.1  …  194.5  212.5  218.9  189.5  178.2
 180.9  177.2  194.6  184.0  190.9     206.9  192.1  222.2  194.5  201.5
 164.2  192.6  201.8  179.8  200.9     220.4  221.1  217.4  211.0  204.1
 168.8  173.4  183.4  158.5  176.1     198.2  190.9  194.8  180.0  177.8
 179.5  192.0  198.5  187.9  203.9     212.5  197.6  228.8  205.6  207.0
 156.9  172.4  188.6  153.1  174.2  …  191.4  184.4  204.4  183.6  176.8
 172.8  182.4  195.9  183.5  194.0     214.1  193.0  217.0  197.8  187.6
 195.0  199.0  213.8  197.0  199.0     219.1  207.6  233.1  209.9  209.8
   ⋮                     